In [ ]:
import os
import re
import numpy as np
import pandas as pd
import cv2

# directories
PRED_DIR = r"C:\Users\User\Desktop\Mirkwood\segmentations"
GT_DIR = r"C:\Users\User\Desktop\Mirkwood\gt"
OUT_CSV = "results.csv"
THRESHOLD = 127  # threshold for binarization (0-255)

# functions
def get_image_number(filename):
    match = re.findall(r'\d+', filename)
    return match[0] if match else None

def binarize(image, threshold=THRESHOLD):
    _, binary = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    return binary

def calculate_metrics(pred, gt):
    pred = pred // 255
    gt = gt // 255

    TP = np.sum((pred == 1) & (gt == 1))
    FP = np.sum((pred == 1) & (gt == 0))
    FN = np.sum((pred == 0) & (gt == 1))
    TN = np.sum((pred == 0) & (gt == 0))

    total = TP + FP + FN + TN

    dice = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0
    iou = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0
    accuracy = (TP + TN) / total if total > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

    return {
        "Dice": dice,
        "IoU": iou,
        "Accuracy": accuracy,
        "Precision": precision,
        "Specificity": specificity
    }

# main code
pred_files = [f for f in os.listdir(PRED_DIR) if f.lower().endswith(('.png', '.tif', '.tiff', '.jpg'))]
gt_files = os.listdir(GT_DIR)

results = []

for pred_name in pred_files:
    num = get_image_number(pred_name)
    match = next((f for f in gt_files if num and num in f), None)

    if not match:
        print(f"No ground truth match for: {pred_name}")
        continue

    # load images
    pred_path = os.path.join(PRED_DIR, pred_name)
    gt_path = os.path.join(GT_DIR, match)

    pred = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
    gt = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

    if pred is None or gt is None:
        print(f"Could not read: {pred_name} or {match}")
        continue

    if pred.shape != gt.shape:
        print(f"Size mismatch: {pred_name} and {match}")
        continue

    # binarize
    pred_bin = binarize(pred)
    gt_bin = binarize(gt)

    # compute metrics
    metrics = calculate_metrics(pred_bin, gt_bin)
    metrics["Image"] = pred_name
    results.append(metrics)

# save to csv
df = pd.DataFrame(results)
df = df[["Image", "Dice", "IoU", "Accuracy", "Precision", "Specificity"]]
df.to_csv(os.path.join(PRED_DIR, OUT_CSV), index=False)
print(f"Saved results to: {os.path.join(PRED_DIR, OUT_CSV)}")